In [1]:
from itertools import combinations
from z3 import *
import numpy as np
import re

In [2]:
def at_least_k(bool_vars, k, name):
    return at_most_k([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k(bool_vars, k, name):
    return And(at_most_k(bool_vars, k, name+"1"), at_least_k(bool_vars, k, name+"2"))

In [3]:
# ---------EXAMPLE DATA ---------
 
# Couriers
m = 3
 
# Packages
n = 7
 
# Load sizes
l = [15, 10, 7]
 
# Size of the package
weights = [3, 2, 6, 8, 5, 4, 4]
 
# x drop point
x = [1, 2, 2, 4, 5, 5, 6, 3]
 
# y drop point
y = [3, 1, 5, 0, 2, 5, 4, 3]
 


In [4]:
# -------READ DATA FROM FILE-----

# Read instance and extract data
lines = []
with open("../in/pdf.dzn") as f: #INSERT INSTANCE FILE HERE
    for line in f:
        line = re.sub("[^0123456789\.\ -]","",line)
        line = line.strip()
        lines.append(line)
# line 1: m
m = int(lines[0])
n = int(lines[1])
l = [int(s) for s in lines[2].split()]
weights = [int(s) for s in lines[3].split()]
x = [int(s) for s in lines[4].split()]
y = [int(s) for s in lines[5].split()]

instance = {'m': m, 'n':n, 'capacities':l, 'weigths':weights, 'instancedx':x, 'instancedy':y}
print(instance)

{'m': 3, 'n': 7, 'capacities': [15, 10, 7], 'weigths': [3, 2, 6, 8, 5, 4, 4], 'instancedx': [1, 2, 2, 4, 5, 5, 6, 3], 'instancedy': [3, 1, 5, 0, 2, 5, 4, 3]}


In [5]:
# --------- FUNCTIONS ---------

x = x[-1:] + x[:-1]
y = y[-1:] + y[:-1]
 

def manhattan_distance(x1, y1, x2, y2):
    distance = 0
    absX = abs(x2 - x1)
    absY = abs(y2 - y1)
    distance = absX + absY
    return distance
 
def create_distance_matrix(listX, listY):
    distance_matrix = np.zeros( (len(listX), len(listX)) )
    for i in range(len(listX)):
        for j in range(len(listX)):
            distance_matrix[i, j] = manhattan_distance(listX[i], listY[i], listX[j], listY[j])
 
    return distance_matrix.astype('int')
 
# --------- VARIABLES ---------
 
distance_matrix = create_distance_matrix(x, y)
print(distance_matrix)

def all_tours_distance(t):
    tot = 0
    for i in range(len(t)-1):
        tot += distance_matrix[t[i], t[i+1]]
    return tot

[[0 2 3 3 4 3 4 4]
 [2 0 3 3 6 5 6 6]
 [3 3 0 4 3 4 7 7]
 [3 3 4 0 7 6 3 5]
 [4 6 3 7 0 3 6 6]
 [3 5 4 6 3 0 3 3]
 [4 6 7 3 6 3 0 2]
 [4 6 7 5 6 3 2 0]]


In [6]:
def intToBool(val, include_tolerance = False, tol=17, name=None):
    if name == None:
        name=str(val)
    clauses = []
    bits = format(val, "b")
    length = len(str(bits))
    if include_tolerance == True:
        if tol < length:
            raise Exception('tol must be greater than the length of the int')
        res = [Bool(f"{name}_{i}") for i in range(tol)]
        bits = '0'*(tol-length) + bits
        length = tol
        
    else: res = [Bool(f"{name}_{i}") for i in range(length)]

    for i in range(length):
        if bits[i] == "1":
            clauses.append(res[i])
        else:
            clauses.append(Not(res[i]))

    return (res, clauses)

def boolToString(model, include_tolerance = False, res_name='res'):
    res = ""
    for i in range(len(model)):
        try:
            if model.evaluate(Bool(f"{res_name}_{i}")):
                res = res + "1"
            else:
                res = res + "0"
        except:
            ""
    if include_tolerance:
        return res
    new_res = ""
    found = False
    for i in range(len(res)):
        if not found and res[i] == "1":
            found = True
        if found:
            new_res = new_res + res[i]
    return new_res

def boolToInt(model, res_name='res'):            
    return int(boolToString(model, res_name=res_name), 2)

def addBools(v1, v2, tolerance = 17, res_name='res'):
    clauses = []
    val1_name = str(v1[0])
    val2_name = str(v2[0])
    if len(v1) != len(v2):
        if len(v1) > len(v2):
            for i in range(len(v1) - len(v2)):
                v2.insert(0, Bool(f"n_{val2_name}_minus_{i}"))
                clauses.append(Not(v2[0]))
        else:
            for i in range(len(v2) - len(v1)):
                v1.insert(0, Bool(f"n_{val1_name}_minus_{i}"))
                clauses.append(Not(v1[0]))
    for i in range(tolerance - len(v1)):
        v1.insert(0, Bool(f"n_{val1_name}_minus_{i}_tollerance"))
        clauses.append(Not(v1[0]))
        v2.insert(0, Bool(f"n_{val2_name}_minus_{i}_tollerance"))
        clauses.append(Not(v2[0]))
    res = [Bool(f"{res_name}_{i}") for i in range(len(v1))]
    carry = [Bool(f"c_{res_name}_{i}") for i in range(len(v1) + 1)]
    for i in range(0, len(v1)):
        clauses.append(((v1[i] == v2[i]) == res[i]) == carry[i+1])
        clauses.append(carry[i] == Or([And(v1[i], v2[i]), And(v1[i], carry[i+1]), And(v2[i], carry[i+1])]))
    clauses.append(And(Not(carry[0]), Not(carry[-1])))
    return (res, clauses)

# i   0 1 2 3 4
# c 0 1 2 3 4
# a   0 1 2 3
# b   0 1 2 3

In [7]:
# paper try to model hamiltonian cycles
# edit it works uwu
# rows are which vertex, col where one is the order (from 1st to last)
visit_order_l = [[Bool(f"visit_{i}_{j}") for j in range(1,n+m+1)] for i in range(1,n+m+1)]
visit_order = np.array(visit_order_l)
vo_len = visit_order.shape[0]
#assert vo_len == m+n
#print(visit_order)
# think is better to reform edges as a single matrix of (n+m)^2 elements where we have m bases.
couriers_edges_l = [[Bool(f"edge_{i}_{j}") for j in range(1,n+m+1)] for i in range(1,n+m+1)]
couriers_edges = np.array(couriers_edges_l)

ce_len = couriers_edges.shape[0]
assert ce_len == vo_len

s = Solver()
# constraints for the couriers edges

# each package appears in exactly one edge (C2)
for row in range(ce_len):
    c2 = exactly_k(couriers_edges[row,:], 1, f'ce_exactly_one_visited_row_{row}')
    s.add(c2)

# each package appears in exactly one edge (C2)
for col in range(ce_len):
    c2_b = exactly_k(couriers_edges[:,col], 1, f'ce_exactly_one_visited_col_{col}')
    s.add(c2_b)

# no edges between the same packages (false on the diagonals)
for i in range(ce_len):
    s.add(Not(couriers_edges[i][i]))

# constraints for the visit orders

# each package location is visited exactly once (C6)
for row in range(vo_len):
    c6 = exactly_k(visit_order[row,:], 1, f'vo_exactly_one_visited_row_{row}')
    s.add(c6)

# each position is related to only one vertex (theoretically duplicate constr)
for col in range(vo_len):
    c6_c = exactly_k(visit_order[:,col], 1, f'vo_exactly_one_visited_col_{col}')
    s.add(c6_c)

# package base of the first courier is visited first
first_base = n
s.add(visit_order[first_base, 0])


# if there is an edge from v1 to vi then vi's position is 2 (C3')
#v1 in our case is the base of the first courier (index n in  the array)
# if there is an edge from vi to v1 then vi's position is last (n+m) (C4')
# index of the first base is n (from 0..n-1 there are packages)
#implies all others are false maybe is more efficient?
for i in range(vo_len):
    if i != first_base:
        c3 = Implies(couriers_edges[first_base, i], visit_order[i,2-1])
        c4 = Implies(couriers_edges[i, first_base], visit_order[i,n+m-1])
        s.add(c3)
        s.add(c4)

# ensures that if edge (i, j)is in the Hamiltonian cycle, and vertex i’s position 
# is p, then vertex j’s position is p + 1. (C5')
for i in range(vo_len):
    for j in range(vo_len):
        for p in range(2, vo_len-1):
            if i != first_base and j != first_base and i!=j:
                c5 = Implies(And(couriers_edges[i,j], visit_order[i,p]), visit_order[j,p+1])
                s.add(c5)
            #else: print('skip', Implies(And(couriers_edges[i,j], visit_order[i,p]), visit_order[j,p+1]))



In [8]:
# constrain the distance
# we have m matrices that are n+1 x n+1, so in total m*n+1*n+1 1s and 0s, where
# we need to sum all the values in the distance matrix corresponding to a 1 (the edge is present)
# so my idea is to make m*n+1*n+1 partial sums all over the matrix cells to finish with the total distance
# if a cell is 1 (edge is present) then the partial sum is part_i = dm(i,j) + part(k-1)
# if a cell is 0 (edge is not present) then the partial sum is part_i = part(k-1)
# total distance == part_last
bit_array_len = 17
partial_sum = []

# addbools: v1, v2 -> solv.add(propositions for sum)
#addbools: v1, v2, res -> solv.add(propositions for sum)

# adds values for distance matrix into the solver
distance_matrix_bool = [[None for _ in  range(n+1)] for _ in range(n+1)]
for i in range (n+1):
    for j in range (n+1):
        res, clauses = intToBool(distance_matrix[i,j], name=f"distance_matrix_{i}_{j}", include_tolerance=True)
        distance_matrix_bool[i][j] = res
        s.add(clauses)


#[111] <=> [111] maybe?
#init of partial sums
partial_sum_prev = [Bool(f"partial_distance_sum_0_{ind}") for ind in range(0, bit_array_len)]
for var in partial_sum_prev:
    s.add(Not(var)) # set to 0b

partial_sum_index = 1

## Total distance using couriers_edges
for i in range(ce_len): #ce_len = n+m
    for j in range(ce_len):
        # this is to manage bases n+1..n+m+1
        i_sum, j_sum = i+1, j+1 #indexes in bool distance matrix are 0+1-n while indexes here are 0-n+m
        if i+1>n:
            i_sum=0
        if j+1>n:
            j_sum=0
        partial_sum_curr, clauses = addBools(distance_matrix_bool[i_sum][j_sum], partial_sum_prev, res_name=f'partial_distance_sum_{partial_sum_index}')
        s.add(Implies(couriers_edges[i][j], And(clauses)))
        for u in range(len(partial_sum_prev)):
            s.add(Implies(Not(couriers_edges[i][j]),partial_sum_curr[u] == partial_sum_prev[u]))
        partial_sum_prev = partial_sum_curr
        partial_sum_index+=1

## Total distance using visit_order
# for dest in range(ce_len):
#     dest_sum = dest
#     if dest+1>n:
#         dest_sum=0
#     partial_sum_curr, clauses = addBools(distance_matrix_bool[dest_sum][0], partial_sum_prev, res_name=f'partial_distance_sum_{partial_sum_index}')
#     s.add(Implies(visit_order[dest, 1], And(clauses)))
# partial_sum_prev = partial_sum_curr
# partial_sum_index+=1

# for visit in range(2, vo_len):
#     for dest in range(ce_len):
#         dest_sum = dest
#         if dest+1>n:
#             dest_sum=0
#         for prev_dest in range(ce_len):
#             prev_dest_sum = prev_dest
#             if prev_dest+1>n:
#                 prev_dest_sum=0
#             if prev_dest != dest:
#                 partial_sum_curr, clauses = addBools(distance_matrix_bool[dest_sum][prev_dest_sum], partial_sum_prev, res_name=f'partial_distance_sum_{partial_sum_index}')
#                 s.add(Implies(And(visit_order[dest, visit], visit_order[prev_dest_sum, visit-1]), And(clauses)))
#     partial_sum_prev = partial_sum_curr
#     partial_sum_index+=1

# for dest in range(ce_len):
#     dest_sum = dest
#     if dest+1>n:
#         dest_sum=0
#     partial_sum_curr, clauses = addBools(distance_matrix_bool[dest_sum][0], partial_sum_prev, res_name=f'partial_distance_sum_{partial_sum_index}')
#     s.add(Implies(visit_order[dest, ce_len-1], And(clauses)))
# partial_sum_prev = partial_sum_curr

total_distance = partial_sum_curr

In [9]:
# block for the weights

def greater_or_equal_boolean_function(b1, b2):
    """returns clauses for bit1 >= b2"""
    clauses = []
    #clauses.append(Implies(b1, Or(b2, Not(b2))))
    clauses.append(Not(And(Not(b1), b2)))

    return clauses

def less_or_equal_boolean_function(b1, b2):
    """returns proposition for bit1 >= b2"""
    #clauses.append(Implies(b1, Or(b2, Not(b2))))
    return Not(And(Not(b2), b1))

def less_or_equal(v1, v2, tolerance = 17):
    """"v1, v2: boolean arrays"""
    #makes v1 the same length as v2
    clauses = []
    val1_name = str(v1[0])
    val2_name = str(v2[0])
    if len(v1) != len(v2):
        if len(v1) > len(v2):
            for i in range(len(v1) - len(v2)):
                v2.insert(0, Bool(f"less_eq_n_{val2_name}_minus_{i}"))
                clauses.append(Not(v2[0]))
        else:
            for i in range(len(v2) - len(v1)):
                v1.insert(0, Bool(f"less_eq_{val1_name}_minus_{i}"))
                clauses.append(Not(v1[0]))
    for i in range(tolerance - len(v1)):
        v1.insert(0, Bool(f"less_eq_{val1_name}_minus_{i}_tollerance"))
        clauses.append(Not(v1[0]))
        v2.insert(0, Bool(f"less_eq_{val2_name}_minus_{i}_tollerance"))
        clauses.append(Not(v2[0]))

    # first bit is 
    clauses.append(less_or_equal_boolean_function(v1[0],v2[0]))
    for i in range(0, len(v1)-1):
        bit_wise_and = []
        for j in range(0, i+1):
            bit_wise_and.append(v1[j]==v2[j])
        clauses.append(Implies(And(bit_wise_and), less_or_equal_boolean_function(v1[i+1],v2[i+1])))
    
    return clauses

In [10]:
# block for capacity constraints of couriers
# each courier has a sum of the packages 
#s = Solver()
#make the weights boolean, and also capacities

# adds values for distance matrix into the solver
print(len(weights))
weights_bool = [None for _ in  range(n)]
for i in range (n):
    res, clauses = intToBool(weights[i], name=f"weights_{i+1}", include_tolerance=True)
    weights_bool[i] = res
    s.add(clauses)

capacities_bool = [None for _ in  range(m)]
for i in range (m):
    res, clauses = intToBool(l[i], name=f"capacities_{i+1}", include_tolerance=True)
    capacities_bool[i] = res
    s.add(clauses)

# delivered by m x n, there a 1 in pos i,j iff i delivers package j
delivered_by = [[Bool(f'{j+1}_delivered_by_{i+1}') for j in range(n)] for i in  range(m)]
delivered_by = np.array(delivered_by)


# exactly 1 in a column (1 package is delivered by only 1 courier)
for col in range(n):
    clauses = exactly_k(delivered_by[:,col], 1, f'delivered_by_exactly_one_{col}')
    s.add(clauses)

# there is a 1 for package i of courier k iff visit order[i, x]. x 

for k in range(m):
    for i in range(n):
        for j in range(n):
            if j!=i:
                #lol = delivered_by[k,i] == (And(delivered_by[k,j], couriers_edges[j,i]))
                lol = Implies((And(delivered_by[k,j], couriers_edges[j,i])), delivered_by[k,i])
                #print(lol)
                s.add(lol)

for k in range(m):
    for i in range(n):
        #lol = delivered_by[k,i] == couriers_edges[n+k,i]
        lol = Implies(couriers_edges[n+k,i], delivered_by[k,i])
        print(lol)
        s.add(lol)

#print(s.check())
#model = s.model()   

#for i in range(m):
#    for j in range(n):
#        print(model.evaluate(Bool(f'{j+1}_delivered_by_{i+1}')), end='\t')
#    print()

7
Implies(edge_8_1, 1_delivered_by_1)
Implies(edge_8_2, 2_delivered_by_1)
Implies(edge_8_3, 3_delivered_by_1)
Implies(edge_8_4, 4_delivered_by_1)
Implies(edge_8_5, 5_delivered_by_1)
Implies(edge_8_6, 6_delivered_by_1)
Implies(edge_8_7, 7_delivered_by_1)
Implies(edge_9_1, 1_delivered_by_2)
Implies(edge_9_2, 2_delivered_by_2)
Implies(edge_9_3, 3_delivered_by_2)
Implies(edge_9_4, 4_delivered_by_2)
Implies(edge_9_5, 5_delivered_by_2)
Implies(edge_9_6, 6_delivered_by_2)
Implies(edge_9_7, 7_delivered_by_2)
Implies(edge_10_1, 1_delivered_by_3)
Implies(edge_10_2, 2_delivered_by_3)
Implies(edge_10_3, 3_delivered_by_3)
Implies(edge_10_4, 4_delivered_by_3)
Implies(edge_10_5, 5_delivered_by_3)
Implies(edge_10_6, 6_delivered_by_3)
Implies(edge_10_7, 7_delivered_by_3)


In [11]:
# sum for check total weight per courier
total_load_per_courier = []

zero_sum = [Bool(f"partial_sum_weights_0_{ind}") for ind in range(0, bit_array_len)]
for var in zero_sum:
    s.add(Not(var)) # set to 0b

for k in range(m):
    partial_sum_index = 1
    partial_sum_prev = zero_sum
    for i in range(n):
        partial_sum_curr, clauses = addBools(weights_bool[i], partial_sum_prev, res_name=f'partial_sum_weights_courier_{k+1}_{partial_sum_index}')
        s.add(Implies(delivered_by[k][i], And(clauses)))
        for u in range(len(partial_sum_prev)):
            s.add(Implies(Not(delivered_by[k,i]),partial_sum_curr[u] == partial_sum_prev[u]))
        partial_sum_prev = partial_sum_curr
        partial_sum_index+=1
    total_load_per_courier.append(partial_sum_curr)

In [12]:
# check capacity of each courier

for k in range(m):
    s.add(less_or_equal(total_load_per_courier[k], capacities_bool[k]))

In [13]:
# manually add constraint on distance
lmao, clauses = intToBool(int(3248/2))
s.add(clauses)
s.add(less_or_equal(total_distance,lmao))

In [14]:
s.check()

sat

In [15]:
model = s.model()
def readVar(var_name, model):
    """returns int value of a given variable name. 0 if not present."""
    res = ''
    for i in range(bit_array_len):
        if model[Bool(f"{var_name}_{i}")]:
            res = res + "1"
        else:
            res = res + "0"
    res_int = int(res,2)
    return res_int
print("Delivered by:")
for i in range(m):
    for j in range(n):
        print(model.evaluate(Bool(f'{j+1}_delivered_by_{i+1}')), end='\t')
    print()

for k in range(m):
    for i in range(0, n):
        print(f"partial sum courier_{k+1} " + str(i) + "= ", str(readVar(f"partial_sum_weights_courier_{k+1}_{i+1}", model)))
        

Delivered by:
True	False	False	True	False	True	False	
False	False	True	False	False	False	True	
False	True	False	False	True	False	False	
partial sum courier_1 0=  3
partial sum courier_1 1=  3
partial sum courier_1 2=  3
partial sum courier_1 3=  11
partial sum courier_1 4=  11
partial sum courier_1 5=  15
partial sum courier_1 6=  15
partial sum courier_2 0=  0
partial sum courier_2 1=  0
partial sum courier_2 2=  6
partial sum courier_2 3=  6
partial sum courier_2 4=  6
partial sum courier_2 5=  6
partial sum courier_2 6=  10
partial sum courier_3 0=  0
partial sum courier_3 1=  2
partial sum courier_3 2=  2
partial sum courier_3 3=  2
partial sum courier_3 4=  7
partial sum courier_3 5=  7
partial sum courier_3 6=  7


In [16]:
#visualizer for enry
solution = s.model()

#courier edges
print('Courier edges')
print('\t', end='')
[print(str(j+1) + "\t", end='') for j in range(ce_len)]
print()
for i in range(ce_len):
    print(i+1, end='\t')
    for j in range(ce_len):
        print(solution.evaluate(couriers_edges[i][j], True), end="\t")
    print("\n")

# visit order
print('Visit order')
print('\t', end='')
[print(str(j+1) + "\t", end='') for j in range(vo_len)]
print()
for i in range(vo_len):
    print(i+1, end='\t')
    for j in range(vo_len):
        print(solution.evaluate(visit_order[i][j], True), end="\t")
    print("\n")

# need a function to print the paths for readability

Courier edges
	1	2	3	4	5	6	7	8	9	10	
1	False	False	False	True	False	False	False	False	False	False	

2	False	False	False	False	False	False	False	True	False	False	

3	False	False	False	False	False	False	False	False	False	True	

4	False	False	False	False	False	True	False	False	False	False	

5	False	True	False	False	False	False	False	False	False	False	

6	False	False	False	False	False	False	False	False	True	False	

7	False	False	True	False	False	False	False	False	False	False	

8	True	False	False	False	False	False	False	False	False	False	

9	False	False	False	False	False	False	True	False	False	False	

10	False	False	False	False	True	False	False	False	False	False	

Visit order
	1	2	3	4	5	6	7	8	9	10	
1	False	True	False	False	False	False	False	False	False	False	

2	False	False	False	False	False	False	False	False	False	True	

3	False	False	False	False	False	False	True	False	False	False	

4	False	False	True	False	False	False	False	False	False	False	

5	False	False	False	False	False	False	False	F

In [17]:
# code to print the paths from the boolean variables
def get_tour(model):
    tour = np.zeros(n+m+1).astype('int')
    for i in range(vo_len):
        for j in range(vo_len):
            if model.evaluate(visit_order[i][j], True):
                #print(i,j, visit_order[i,j]) # add +1 to i, j to get package and position in 1..n+m
                tour[j] = i+1
    tour[-1] = 0
    return tour


def tour_to_string(tour):
    tour_string = '['
    for p in tour:
        if p > n: #transform all bases into the same origin
            tour_string += '0'
        else: tour_string += str(p)
        tour_string += ', '
    tour_string = tour_string[0:-2] # remove unneeded ', '
    tour_string += ']' # close array
    return tour_string

def tour_to_string_couriers(tour):
    tour_list = ['[']*m
    courier = 0
    for p in tour:
        if p > n: #transform all bases into the same origin. when base, switch courier
            if p != n+1:
                tour_list[courier] += '0, '
                courier += 1 # start filling next list
            tour_list[courier] += '0'
        else: tour_list[courier] += str(p)
        tour_list[courier] += ', '
    
    for courier in range(m):
        tour_list[courier] = tour_list[courier][0:-2] # remove unneeded ', '
        tour_list[courier] += ']' # close array
    return tour_list

def tour_to_plot_string(tour):
    tour_list = [ [] for k in range(m)]
    courier = 0
    for p in tour:
        if p > n: #transform all bases into the same origin. when base, switch courier
            if p != n+1:
                tour_list[courier].append(0)
                courier += 1 # start filling next list
            tour_list[courier].append(0)
        else: tour_list[courier].append(p)
    return tour_list    

tour = get_tour(solution)
print(get_tour(solution))
print(tour_to_string(tour))
print(tour_to_string_couriers(tour))
print(tour_to_plot_string(tour))

[ 8  1  4  6  9  7  3 10  5  2  0]
[0, 1, 4, 6, 0, 7, 3, 0, 5, 2, 0]
['[0, 1, 4, 6, 0]', '[0, 7, 3, 0]', '[0, 5, 2, 0]']
[[0, 1, 4, 6, 0], [0, 7, 3, 0], [0, 5, 2, 0]]


In [18]:
#print(boolToString(solution, res_name=f'partial_sum_15'))
model = s.model()
#print(s.assertions()[-20:])

print(str(total_distance[0])[:-2])
var_name = str(total_distance[0])[:-2]
for i in range(0, ce_len**2):
       print("partial sum" + str(i) + "= ", str(readVar(f"partial_distance_sum_{i}", model)))
print("total distance: ", str(readVar(str(total_distance[0])[:-2], model)))
print("real total distance:", all_tours_distance([i if i<=n else 0 for i in get_tour(model) ]))

partial_distance_sum_100
partial sum0=  0
partial sum1=  0
partial sum2=  0
partial sum3=  0
partial sum4=  6
partial sum5=  6
partial sum6=  6
partial sum7=  6
partial sum8=  6
partial sum9=  6
partial sum10=  6
partial sum11=  6
partial sum12=  6
partial sum13=  6
partial sum14=  6
partial sum15=  6
partial sum16=  6
partial sum17=  6
partial sum18=  9
partial sum19=  9
partial sum20=  9
partial sum21=  9
partial sum22=  9
partial sum23=  9
partial sum24=  9
partial sum25=  9
partial sum26=  9
partial sum27=  9
partial sum28=  9
partial sum29=  9
partial sum30=  12
partial sum31=  12
partial sum32=  12
partial sum33=  12
partial sum34=  12
partial sum35=  12
partial sum36=  18
partial sum37=  18
partial sum38=  18
partial sum39=  18
partial sum40=  18
partial sum41=  18
partial sum42=  22
partial sum43=  22
partial sum44=  22
partial sum45=  22
partial sum46=  22
partial sum47=  22
partial sum48=  22
partial sum49=  22
partial sum50=  22
partial sum51=  22
partial sum52=  22
partial 